In [1]:
import requests
import json

def setup_api():
    endpoint = "https://linkedin-data-api.p.rapidapi.com/get-profile-data-by-url"
    querystring = {"url":"https://www.linkedin.com/in/adamselipsky/"}
    headers = {
        "x-rapidapi-key": "b1eae6d022msh7daf5d27f9b259dp1e9c79jsn92395fddbca6",
        "x-rapidapi-host": "linkedin-data-api.p.rapidapi.com"
    }
    return endpoint, querystring, headers

endpoint, querystring, headers = setup_api()

def make_request():
    response = requests.get(endpoint, headers=headers, params=querystring)
    if response.status_code == 429: 
        print("Rate limit hit")
        time.sleep(60) 
        return make_request()
    return response

response = make_request()
print(response.json())

{'id': 11848598, 'urn': 'ACoAAAC0y5YB1d3L356Yaf3g3Tb5PR4O7scFb5o', 'username': 'adamselipsky', 'firstName': 'Adam', 'lastName': 'Selipsky', 'isCreator': True, 'isOpenToWork': False, 'isHiring': False, 'profilePicture': 'https://media.licdn.com/dms/image/v2/D5603AQGZJ-njdtv_-g/profile-displayphoto-shrink_800_800/profile-displayphoto-shrink_800_800/0/1719507065641?e=1738195200&v=beta&t=xKE5RqNLfdf8i3X9lqSFa_x1Lq4e1HcXzwFy4pEIhlM', 'backgroundImage': [{'width': 266, 'height': 200, 'url': 'https://media.licdn.com/dms/image/v2/D5616AQEetlwRti-pwA/profile-displaybackgroundimage-shrink_200_800/profile-displaybackgroundimage-shrink_200_800/0/1719506391020?e=1738195200&v=beta&t=SGX6QFtPh962ZISjR301s4yj565SwXmhbbOrrc_j7Dw'}, {'width': 466, 'height': 350, 'url': 'https://media.licdn.com/dms/image/v2/D5616AQEetlwRti-pwA/profile-displaybackgroundimage-shrink_350_1400/profile-displaybackgroundimage-shrink_350_1400/0/1719506391020?e=1738195200&v=beta&t=M2yuFBcavnRZ9zpulzCURLLUD_qDZuXyl0ID3FCBR8o'}], 

In [2]:
def fetch_data(endpoint, querystring, headers):
    try:
        response = requests.get(endpoint, headers=headers, params=querystring) 
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            print("Rate limit hit. Retrying after a delay...")
            time.sleep(60)
            return fetch_data(endpoint, querystring, headers)
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error during request: {e}")
        return None

In [3]:
def process_data(data):
    profile = {
        'id': data.get('id', 'N/A'),
        'urn': data.get('urn', 'N/A'),
        'username': data.get('username', 'N/A'),
        'full_name': f"{data.get('firstName', '')} {data.get('lastName', '')}".strip(),
        'is_creator': data.get('isCreator', False),
        'is_open_to_work': data.get('isOpenToWork', False),
        'is_hiring': data.get('isHiring', False),
        'profile_picture': data.get('profilePicture', 'N/A'),
        'headline': data.get('headline', 'N/A'),
        'summary': data.get('summary', 'N/A'),
        'location': data.get('geo', {}).get('full', 'N/A'),
        'languages': data.get('languages', 'N/A'),
    }
    educations = [
        {
            'school_name': edu.get('schoolName', 'N/A'),
            'degree': edu.get('degree', 'N/A'),
            'field_of_study': edu.get('fieldOfStudy', 'N/A'),
            'start_year': edu.get('start', 'N/A'),
            'end_year': edu.get('end', 'N/A'),
            'school_url': edu.get('url', 'N/A'),
        }
        for edu in data.get('educations', [])
    ]
    positions = [
        {
            'company_name': pos.get('companyName', 'N/A'),
            'title': pos.get('title', 'N/A'),
            'location': pos.get('location', 'N/A'),
            'start_date': pos.get('start', 'N/A'),
            'end_date': pos.get('end', 'N/A'),
            'company_url': pos.get('companyURL', 'N/A'),
            'company_logo': pos.get('companyLogo', 'N/A'),
            'description': pos.get('description', 'N/A'),
        }
        for pos in data.get('position', [])
    ]
    skills = [
        {
            'skill_name': skill.get('name', 'N/A'),
            'endorsements': skill.get('endorsementsCount', 0),
        }
        for skill in data.get('skills', [])
    ]
    processed_data = {
        'profile': profile,
        'educations': educations,
        'positions': positions,
        'skills': skills,
    }
    return processed_data

In [4]:
from rich.console import Console
from rich.table import Table
from rich.json import JSON

console = Console()

def display_results():
    endpoint, querystring, headers = setup_api()
    response_data = fetch_data(endpoint, querystring, headers)
    if response_data:
        processed_data = process_data(response_data)
        console.print("[bold underline cyan]Profile Details[/]")
        profile = processed_data['profile']
        console.print(f"[bold]Full Name:[/bold] {profile['full_name']}")
        console.print(f"[bold]Headline:[/bold] {profile['headline']}")
        console.print(f"[bold]Location:[/bold] {profile['location']}")
        console.print(f"[bold]Summary:[/bold] {profile['summary']}")
        console.print(f"[bold]Profile Picture URL:[/bold] {profile['profile_picture']}")
        console.print("\n")
        console.print("[bold underline green]Education Details[/]")
        education_table = Table(title="Education Details", show_header=True, header_style="bold magenta")
        education_table.add_column("School Name", style="dim")
        education_table.add_column("Degree")
        education_table.add_column("Field of Study")
        education_table.add_column("Years", justify="center")
        for edu in processed_data['educations']:
            education_table.add_row(
                edu['school_name'],
                edu['degree'],
                edu['field_of_study'],
                f"{edu['start_year']} - {edu['end_year']}"
            )
        console.print(education_table)
        console.print("\n")
        console.print("[bold underline yellow]Work Experience[/]")
        position_table = Table(title="Work Experience", show_header=True, header_style="bold blue")
        position_table.add_column("Company Name")
        position_table.add_column("Title")
        position_table.add_column("Location")
        position_table.add_column("Duration", justify="center")
        position_table.add_column("Description", overflow="fold")
        for pos in processed_data['positions']:
            position_table.add_row(
                pos['company_name'],
                pos['title'],
                pos['location'],
                f"{pos['start_date']} - {pos['end_date']}",
                pos['description']
            )
        console.print(position_table)
        console.print("\n")
        console.print("[bold underline blue]Skills[/]")
        skills_table = Table(title="Skills", show_header=True, header_style="bold cyan")
        skills_table.add_column("Skill Name")
        skills_table.add_column("Endorsements", justify="center")
        for skill in processed_data['skills']:
            skills_table.add_row(
                skill['skill_name'],
                str(skill['endorsements'])
            )
        console.print(skills_table)
        console.print("\n")

In [5]:
display_results()

Profile Details

Full Name: Adam Selipsky

Headline: Former CEO at Amazon Web Services (AWS). Former President and CEO at Tableau.

Location: Greater Seattle Area

Summary: Adam Selipsky is the CEO of Amazon Web Services (AWS), the world’s most comprehensive and broadly adopted 
cloud platform. Having previously led AWS marketing, sales, and support for 11 years, from 2005-2016, Selipsky 
helped launch and grow AWS from a start-up into a multi-billion dollar business. Prior to rejoining AWS in 2021, 
Selipsky was most recently President and CEO of Tableau Software. He led Tableau through its acquisition by 
Salesforce, in what was the third largest software industry acquisition at the time.

...

                                                 Education Details                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ School Name             ┃ Degree ┃ Field of Study ┃                            Years                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Harvard Business School │ MBA    │                │ {'year': 0, 'month': 0, 'day': 0} - {'year': 0, 'month': 0, │
│                         │        │                │                          'day': 0}                          │
│ Harvard University      │ A.B.   │ Government     │ {'year': 0, 'month': 0, 'day': 0} - {'year': 0, 'month': 0, │
│                         │        │                │                          'day': 0}                          │
└─────────────────────────┴────────┴────────────────┴─────────────────────────────────────────────────────────────┘

Work Experience

                                                  Work Experience                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Company Name           ┃ Title                  ┃ Location    ┃        Duration        ┃ Description            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Amazon Web Services    │ CEO                    │             │     {'year': 2021,     │ Led AWS from           │
│ (AWS)                  │                        │             │ 'month': 5, 'day': 0}  │ pre-revenue to $100B   │
│                        │                        │             │    - {'year': 2024,    │ revenue run-rate with  │
│                        │                        │             │ 'month': 6, 'day': 0}  │ 37% operating margin.  │
│                        │                        │             │                        │ AWS is the world’s     │
│                        │                        │             │                        │ most comprehensive and │
│                        │                        │             │                        │ broadly adopted cloud  │
│                        │                        │             │                        │ offering.              │
│ Tableau Software       │ CEO                    │ Seattle, WA │     {'year': 2016,     │ Transformed Tableau to │
│                        │                        │             │ 'month': 0, 'day': 0}  │ subscription model     │
│                        │                        │             │    - {'year': 2021,    │ from perpetual         │
│                        │                        │             │ 'month': 5, 'day': 0}  │ licensing. Led company │
│                        │                        │             │                        │ through its            │
│                        │                        │             │                        │ acquisition by         │
│                        │                        │             │                        │ Salesforce, in what    │
│                        │                        │             │                        │ was the third largest  │
│                        │                        │             │                        │ software industry      │
│                        │                        │             │                        │ acquisition at the     │
│                        │                        │             │                        │ time. Tableau helps    │
│                        │                        │             │                        │ people see and         │
│                        │                        │             │                        │ understand data.       │
│                        │                        │             │                        │ People use Tableau’s   │
│                        │                        │             │                        │ intuitive business     │
│                        │                        │             │                        │ analytics software to  │
│                        │                        │             │                        │ fuel exploration, ask  │
│                        │                        │             │                        │ questions, and share   │
│                        │                        │             │                        │ insights across their  │
│                        │                        │             │                        │ organizations.         │
│ Amazon Web Services    │ Vice President of      │ Seattle, WA │     {'year': 2005,     │ Worldwide              │
│ (AWS)                  │ Marketing, Sales, &    │             │ 'month': 0, 'day': 0}  │ responsibility for AWS │
│                        │ Support - COO role     │     

Skills

                 Skills                  
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Skill Name             ┃ Endorsements ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Entrepreneurship       │      12      │
│ Product Marketing      │      7       │
│ Business Strategy      │      8       │
│ Mergers & Acquisitions │      5       │
│ Management             │      13      │
│ Marketing Strategy     │      9       │
│ SaaS                   │      5       │
│ Business Development   │      21      │
│ Product Management     │      28      │
│ Management Consulting  │      3       │
│ Program Management     │      4       │
│ Marketing              │      5       │
│ Cloud Computing        │      22      │
│ Go-to-market Strategy  │      9       │
│ Sales                  │      2       │
│ Leadership             │      5       │
│ Start-ups              │      5       │
│ Strategy               │      8       │
│ Strategic Partnerships │      22      │
└────────────────────────┴──────────────┘